In [1]:
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import shutil
from src.label_pipeline import InteriorImageClassifier, generate_class_label2ref_images
from config import (
    MODEL_NAME,
    BASE_PROMPT_EN,
    BASE_PROMPT_RU,
    TRASH_FILTER_PROMPT_EN,
    TRASH_FILTER_PROMPT_RU,
    A0_ONLY_FILTER_PROMPT_EN,
    A0_ONLY_FILTER_PROMPT_RU,
    A1_ONLY_FILTER_PROMPT_RU,
    A1_ONLY_FILTER_PROMPT_EN,
    B1_ONLY_FILTER_PROMPT_EN,
    B1_ONLY_FILTER_PROMPT_RU,
    D0_ONLY_FILTER_PROMPT_EN,
    D0_ONLY_FILTER_PROMPT_RU,
    D1_ONLY_FILTER_PROMPT_EN,
    D1_ONLY_FILTER_PROMPT_RU,
    D0_VS_D1_VS_UNKNOWN_PROMPT_EN_PART1,
    D0_VS_D1_VS_UNKNOWN_PROMPT_EN_PART2
)

In [3]:
def process_directory(
        classifier: InteriorImageClassifier,
        image_dir: Path,
        output_dir: Path,
        ref_images_dir: Path | None = None,
        extensions: tuple = (".jpg", ".png", ".jpeg")
    ) -> None:
    """
    Обрабатывает изображения и сортирует по папкам классов
    """
    output_dir.mkdir(parents=True, exist_ok=True)  # Создаем корневую директорию для результатов
    
    # Обрабатываем каждое изображение
    img_files = [f for f in image_dir.iterdir() if f.suffix.lower() in extensions]
    for img_file in tqdm(img_files, desc="Обработка изображений"):
        class_label2ref_images = None
        if ref_images_dir is not None:
            class_label2ref_images = generate_class_label2ref_images(
                ref_images_dirpath=ref_images_dir,
                max_files_per_class=1
            )
        image = Image.open(img_file)
        result = classifier(image, class_label2ref_images=class_label2ref_images)
        
        class_dir = output_dir / result["class"]  # Создаем папку класса (A0, A1... ERROR)
        class_dir.mkdir(exist_ok=True)
        
        shutil.copy2(img_file, class_dir / img_file.name)  # Копируем изображение в папку класса
    
    print(f"\nГотово! Результаты в {output_dir}")


In [4]:
def process_directory2_for_filtering(
        classifier: InteriorImageClassifier,
        image_dir: Path,
        output_dir: Path,
        custom_prompt: str| None = None,
        ref_images_dir: Path | None = None,
        extensions: tuple = (".jpg", ".png", ".jpeg")
    ) -> None:
    """
    Обрабатывает изображения и сортирует по папкам классов
    """
    output_dir.mkdir(parents=True, exist_ok=True)  # Создаем корневую директорию для результатов
    
    # Обрабатываем каждое изображение
    img_files = [f for f in image_dir.iterdir() if f.suffix.lower() in extensions]
    for img_file in tqdm(img_files, desc="Обработка изображений"):
        class_label2ref_images = None
        if ref_images_dir is not None:
            class_label2ref_images = generate_class_label2ref_images(
                ref_images_dirpath=ref_images_dir,
                max_files_per_class=1
            )
        image = Image.open(img_file)
        result = classifier(image, custom_prompt=custom_prompt)
        
        if result["class"] != "D0":  # trash
            shutil.move(img_file, output_dir)
        else:
            raw_response = result['raw_response'].split()
            if len(raw_response) >= 2:
                confidence = raw_response[1]
                if confidence.isdigit() and int(confidence) <= 8:
                    shutil.move(img_file, output_dir)
    
    print(f"\nГотово! Результаты в {output_dir}")

In [3]:
# Инициализация классификатора
classifier = InteriorImageClassifier(model_name=MODEL_NAME, base_prompt=BASE_PROMPT_EN)
print(classifier.pipe.torch_dtype)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
Device set to use cuda:0


torch.bfloat16


In [ ]:
current_dir = Path.cwd()
print(f"Директория: {current_dir}")
data_dir = current_dir / "data"

image_dir = data_dir / "cian_data/D0"
output_dir = data_dir / "cian_data/D0_uncertain"

process_directory2_for_filtering(
    classifier=classifier,
    image_dir=image_dir,
    output_dir=output_dir,
    custom_prompt=D0_ONLY_FILTER_PROMPT_EN
)

In [ ]:
current_dir = Path.cwd()
print(f"Директория: {current_dir}")
data_dir = current_dir / "data"

image_dir = data_dir / "cosmetic/cosmetic-"
output_dir = data_dir / "labeled_with_few_shot"
ref_images_dir = data_dir / "reference_images_resized"

process_directory(
    classifier=classifier,
    image_dir=image_dir,
    output_dir=output_dir,
    ref_images_dir=ref_images_dir
)

In [5]:
current_dir = Path.cwd()
print(f"Директория: {current_dir}")
data_dir = current_dir / "data"
interior_dataset_dir = data_dir / "interior_dataset"
interior_dataset_dir_D0_cian = interior_dataset_dir / "D0_cian"

output_dir_D0_cian_D0 = interior_dataset_dir / "D0_cian_D0"
output_dir_D0_cian_D0.mkdir(exist_ok=True)
output_dir_D0_cian_D0_confident = output_dir_D0_cian_D0 / "confident"
output_dir_D0_cian_D0_confident.mkdir(exist_ok=True)
output_dir_D0_cian_D0_uncertain = output_dir_D0_cian_D0 / "uncertain"
output_dir_D0_cian_D0_uncertain.mkdir(exist_ok=True)

output_dir_D0_cian_D1 = interior_dataset_dir / "D0_cian_D1"
output_dir_D0_cian_D1.mkdir(exist_ok=True)
output_dir_D0_cian_D1_confident = output_dir_D0_cian_D1 / "confident"
output_dir_D0_cian_D1_confident.mkdir(exist_ok=True)
output_dir_D0_cian_D1_uncertain = output_dir_D0_cian_D1 / "uncertain"
output_dir_D0_cian_D1_uncertain.mkdir(exist_ok=True)


output_dir_D0_cian_unknown = interior_dataset_dir / "D0_cian_unknown"
output_dir_D0_cian_unknown.mkdir(exist_ok=True)

ref_images_dir = data_dir / "reference_images" / "resized_448x448"
ref_images_dir_D0_new = ref_images_dir / "D0_new"
ref_images_dir_D1_new = ref_images_dir / "D1_new"
max_examples_per_class = 10



# Начало промпта
content = [
    {"type": "text", "text": D0_VS_D1_VS_UNKNOWN_PROMPT_EN_PART1},
    {"type": "text", "text": "\nReference examples:"}
]

class_label2ref_images = {'D0': []}
for p in ref_images_dir_D0_new.iterdir():
    class_label2ref_images['D0'].append(Image.open(p))

for class_label, images in class_label2ref_images.items():
    if images:
        selected_images = images[:max_examples_per_class]
        # Добавляем в контент
        for image in selected_images:
            content.extend([
                {"type": "image", "image": image},
                {"type": "text", "text": f"Class: {class_label}"}
            ])


# Добавляем вторую часть промпта
content.extend(
    [
        {"type": "text", "text": D0_VS_D1_VS_UNKNOWN_PROMPT_EN_PART2},
        {"type": "text", "text": "\nReference examples:"}
    ]
)

class_label2ref_images = {'D1': []}
for p in ref_images_dir_D1_new.iterdir():
    class_label2ref_images['D1'].append(Image.open(p))

for class_label, images in class_label2ref_images.items():
    if images:
        selected_images = images[:max_examples_per_class]
        # Добавляем в контент
        for image in selected_images:
            content.extend([
                {"type": "image", "image": image},
                {"type": "text", "text": f"Class: {class_label}"}
            ])


moved = 0
extensions = (".jpg", ".png", ".jpeg")
for img_path in tqdm(sorted(interior_dataset_dir_D0_cian.iterdir())):
    if img_path.suffix.lower() not in extensions:
        continue
    
    # Добавляем целевое изображение
    target_content = [
        {"type": "text", "text": "\nNow classify THIS image:"},
        {"type": "image", "image": Image.open(img_path)},
        {"type": "text", "text": "Answer ONLY with class label:"}
    ]
    messages = [{"role": "user", "content": content + target_content}]
    raw_response = classifier._get_model_response(inputs=messages, max_new_tokens=10)
    predicted_class = classifier.parse_model_response(raw_response)

    if predicted_class == "D0":
        raw_response = raw_response.split()
        if len(raw_response) >= 2:  # Example: "<class_label> <confidence>""
            confidence = raw_response[1]
            if confidence.isdigit():
                if int(confidence) >= 8:
                    shutil.move(img_path, output_dir_D0_cian_D0_confident)
                else:
                    shutil.move(img_path, output_dir_D0_cian_D0_uncertain)
        else:  # trash
            shutil.move(img_path, output_dir_D0_cian_D0_uncertain)

    elif predicted_class == "D1":
        raw_response = raw_response.split()
        if len(raw_response) >= 2:  # Example: "<class_label> <confidence>""
            confidence = raw_response[1]
            if confidence.isdigit():
                if int(confidence) >= 8:
                    shutil.move(img_path, output_dir_D0_cian_D1_confident)
                else:
                    shutil.move(img_path, output_dir_D0_cian_D1_uncertain)
        else:
            shutil.move(img_path, output_dir_D0_cian_D1_uncertain)
    else:  # trash
        shutil.move(img_path, output_dir_D0_cian_unknown)
    
    moved += 1

print(f"Total moved: {moved}")

Директория: /home/little-garden/CodeProjects/InteriorClass


100%|██████████| 8509/8509 [7:17:51<00:00,  3.09s/it]  

Total moved: 8509


In [ ]:
current_dir = Path.cwd()
print(f"Директория: {current_dir}")
data_dir = current_dir / "data"
cian_data_dir = data_dir / "cian_data"
cian_data_dir_D0 = cian_data_dir / "D0"
output_dir = cian_data_dir / "D0_uncertain"
output_dir.mkdir(exist_ok=True)

ref_images_dir = data_dir / "reference_images" / "resized_448x448"
ref_images_dir_D0 = ref_images_dir / "D0"
max_examples_per_class = 10

class_label2ref_images = {'D0': []}
for p in ref_images_dir_D0.iterdir():
    class_label2ref_images['D0'].append(Image.open(p))


# Начало промпта
content = [
    {"type": "text", "text": D0_ONLY_FILTER_PROMPT_EN},
    {"type": "text", "text": "\nReference examples:"}
]


for class_label, images in class_label2ref_images.items():
    if images:
        selected_images = images[:max_examples_per_class]
        # Добавляем в контент
        for image in selected_images:
            content.extend([
                {"type": "image", "image": image},
                {"type": "text", "text": f"Class: {class_label}"}
            ])

print(cian_data_dir_D0)
moved = 0
extensions = (".jpg", ".png", ".jpeg")
for img_path in tqdm(sorted(cian_data_dir_D0.iterdir())):
    if img_path.suffix.lower() not in extensions:
        continue
    
    # Добавляем целевое изображение
    target_content = [
        {"type": "text", "text": "\nNow classify THIS image:"},
        {"type": "image", "image": Image.open(img_path)},
        {"type": "text", "text": "Answer ONLY with class label:"}
    ]
    messages = [{"role": "user", "content": content + target_content}]
    raw_response = classifier._get_model_response(inputs=messages, max_new_tokens=10)
    predicted_class = classifier.parse_model_response(raw_response)

    if predicted_class != "D0":  # trash
        shutil.move(img_path, output_dir)
        moved += 1
    else:
        raw_response = raw_response.split()
        if len(raw_response) >= 2:
            confidence = raw_response[1]
            if confidence.isdigit() and int(confidence) <= 8:
                shutil.move(img_path, output_dir)
                moved += 1
print(f"Total moved: {moved}")

In [ ]:
current_dir = Path.cwd()
print(f"Директория: {current_dir}")
data_dir = current_dir / "data"
cian_data_dir = data_dir / "cian_data"
cian_data_dir_A1 = cian_data_dir / "A1"

output_dir_A1_confident = cian_data_dir / "A1_confident"
output_dir_A1_confident.mkdir(exist_ok=True)

output_dir_A1_uncertain = cian_data_dir / "A1_uncertain"
output_dir_A1_uncertain.mkdir(exist_ok=True)

output_dir_A1_unknown = cian_data_dir / "A1_unknown"
output_dir_A1_unknown.mkdir(exist_ok=True)

ref_images_dir = data_dir / "reference_images" / "resized_448x448"
ref_images_dir_A1 = ref_images_dir / "A1"
max_examples_per_class = 10

class_label2ref_images = {'A1': []}
for p in ref_images_dir_A1.iterdir():
    class_label2ref_images['A1'].append(Image.open(p))


# Начало промпта
content = [
    {"type": "text", "text": A1_ONLY_FILTER_PROMPT_EN},
    {"type": "text", "text": "\nReference examples:"}
]


for class_label, images in class_label2ref_images.items():
    if images:
        selected_images = images[:max_examples_per_class]
        # Добавляем в контент
        for image in selected_images:
            content.extend([
                {"type": "image", "image": image},
                {"type": "text", "text": f"Class: {class_label}"}
            ])

print(cian_data_dir_A1)
moved = 0
extensions = (".jpg", ".png", ".jpeg")
for img_path in tqdm(sorted(cian_data_dir_A1.iterdir())):
    if img_path.suffix.lower() not in extensions:
        continue
    
    # Добавляем целевое изображение
    target_content = [
        {"type": "text", "text": "\nNow classify THIS image:"},
        {"type": "image", "image": Image.open(img_path)},
        {"type": "text", "text": "Answer ONLY with class label:"}
    ]
    messages = [{"role": "user", "content": content + target_content}]
    raw_response = classifier._get_model_response(inputs=messages, max_new_tokens=10)
    predicted_class = classifier.parse_model_response(raw_response)

    if predicted_class == "A1": 
        raw_response = raw_response.split()
        if len(raw_response) >= 2:  # Example: "<class_label> <confidence>""
            confidence = raw_response[1]
            if confidence.isdigit():
                if int(confidence) >= 8:
                    shutil.move(img_path, output_dir_A1_confident)
                else:
                    shutil.move(img_path, output_dir_A1_uncertain)
                
                moved += 1
    else:  # trash
        shutil.move(img_path, output_dir_A1_unknown)
        moved += 1

print(f"Total moved: {moved}")